In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from dfply import *

In [2]:
import psycopg2

conn = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password="June3060!!",
    database = "mimic_data",
    port="5432"
)

cursor = conn.cursor()
print("database connected")

database connected


In [3]:
df = pd.read_csv("../Data/reintubation_df_240127.csv", index_col="Unnamed: 0")

In [4]:
df2 = df.copy()

------

In [5]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_derived.sapsii;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
sapsii = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)
sapsii

경과 시간 :  0.9087834358215332


,subject_id,hadm_id,stay_id,starttime,endtime,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,...,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
0,14046553,20000094,35605481,2150-03-02 15:19:31,2150-03-03 15:19:31,98,0.982729,18,11.0,13.0,...,11.0,10.0,0.0,0.0,0.0,3.0,0.0,26.0,0,6
1,14990224,20000147,30503572,2121-08-30 16:33:54,2121-08-31 16:33:54,35,0.166523,15,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
2,17913090,20000351,30593599,2145-06-13 20:10:27,2145-06-14 20:10:27,24,0.058153,7,0.0,0.0,...,11.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,6
3,16788749,20000808,30610654,2180-01-17 19:32:10,2180-01-18 19:32:10,17,0.025876,7,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,8
4,16788749,20000808,35191063,2180-01-12 22:07:00,2180-01-13 22:07:00,26,0.071716,7,0.0,0.0,...,11.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,14641622,29999186,32199257,2135-07-12 01:51:00,2135-07-13 01:51:00,30,0.106398,7,0.0,5.0,...,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0,8
73177,10053207,29999444,31112026,2199-12-14 19:52:26,2199-12-15 19:52:26,26,0.071716,7,4.0,0.0,...,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0,6
73178,13478841,29999498,33874605,2188-05-13 12:03:00,2188-05-14 12:03:00,34,0.152870,16,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,9,6
73179,10355856,29999625,36975675,2157-11-07 11:51:00,2157-11-08 11:51:00,32,0.128048,12,0.0,5.0,...,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0,8


In [6]:
def group_df_sapsii(group):

    # group의 index,와 row를 하나씩 가져온다.
    for idx, row in group.iterrows():

        # sapsii의 subject_id, hadm_id가 row의 subject_id, hadm_id를 각각 만족하는 sapsii_df를 만든다
        sapsii_df = sapsii[(sapsii["subject_id"] == row["subject_id"]) & (sapsii["hadm_id"] == row["hadm_id"])]

        # row의 int_stayid가 만약 null 값이 아니라면
        if pd.notnull(row["int_stayid"]):

            # sapsii_df의 stay_id와 row의 int_stayid가 똑같은 값을 찾는다.
            sapsii_idx = sapsii_df[sapsii_df["stay_id"] == row["int_stayid"]].first_valid_index()

            # 해당 idx에 할당한다.
            group.loc[idx, "sapsii"] = sapsii_df.loc[sapsii_idx, "sapsii"]
        else:
            sapsii_idx = sapsii_df[sapsii_df["stay_id"] == row["ext_stayid"]].first_valid_index()
            group.loc[idx, "sapsii"] = sapsii_df.loc[sapsii_idx, "sapsii"]
    return group

In [7]:
df_sapsii = df2.groupby(["subject_id", "hadm_id"], group_keys=False).apply(group_df_sapsii)
df_sapsii

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime,sapsii
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN,44.0
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN,83.0
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,NaN,NaN,43.0
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,3930.0,2144-01-30 13:00:00,30.0,65.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,NaN,NaN,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,8416.0,2126-05-05 18:20:00,4430.0,55.0
9004,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,4269.0,NaN,NaN,55.0
9005,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,4201.0,NaN,NaN,44.0
9006,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,1155.0,NaN,NaN,20.0


In [9]:
df_sapsii["sapsii"].isna().sum()

0

# 내용 정리
* sapsii 추출 
* 함수를 사용하여 데이터프레임에 붙여준다.
* 결측치는 없는 것으로 확인됨